# Pré-execução

Rode esta seção para baixar os dados de entrada

In [30]:
!git clone https://github.com/azhow/Machine_Learning.git

fatal: destination path 'Machine_Learning' already exists and is not an empty directory.


## Carregamento dos dados de entrada

In [31]:
# Reads the data from CSV
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
import graphviz
from sklearn.metrics import accuracy_score
import csv

INPUT_PATH = "./Machine_Learning/DT/diabetes.csv"

dataframe = pd.read_csv(INPUT_PATH, header=0)
x, y = dataframe.iloc[:, :-1], dataframe.iloc[:, [-1]]

# Experimento A

Abaixo se encontra o código utilizado para gerar os dados para o experimento A.
São produzidos os seguintes arquivos:

    * baseline.csv - contém o resultado de todas execuções tanto para a implementação que utiliza a medida Gini quanto para a medida de entropia.

    * ENTROPY_BASELINE.png - contém a imagem da árvore de decisão do algoritmo que utiliza a medida de entropia.
    
    * GINI_BASELINE.png - contém a imagem da árvore de decisão da medida Gini. 

## Parâmetros de execução

Aqui podes definir os parâmetros para a execução dos experimentos

In [2]:
# Consts declaration
GINI_BASELINE_GRAPH_FILE = "./Machine_Learning/DT/GINI_BASELINE.dot"
ENTROPY_BASELINE_GRAPH_FILE = "./Machine_Learning/DT/ENTROPY_BASELINE.dot"
BASELINE_EXPERIMENTS = "./Machine_Learning/DT/baseline.csv"
NUM_OF_RUNS = 100

## Execução dos experimentos e exportação para arquivo CSV

In [4]:
# Runs the data the defined number of times and saves the run results
run_results = [["gini", "entropy"]]

for i in range(NUM_OF_RUNS):
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(x), np.array(y).squeeze(), train_size=0.8)
    
    classifier_gini = tree.DecisionTreeClassifier(criterion='gini')
    classifier_gini = classifier_gini.fit(X_train, Y_train)
    with open(GINI_BASELINE_GRAPH_FILE, 'w') as out_file:
        dot_data = tree.export_graphviz(classifier_gini, feature_names=x.columns, out_file=out_file)
    
    classifier_entropy = tree.DecisionTreeClassifier(criterion='entropy')
    classifier_entropy = classifier_entropy.fit(X_train, Y_train)
    with open(ENTROPY_BASELINE_GRAPH_FILE, 'w') as out_file:
        dot_data = tree.export_graphviz(classifier_entropy, feature_names=x.columns, out_file=out_file)
    
    Y_pred_gini = classifier_gini.predict(X_test)
    Y_pred_entropy = classifier_entropy.predict(X_test)

    run_results.append([accuracy_score(Y_test, Y_pred_gini), accuracy_score(Y_test, Y_pred_entropy)])  

# Save experiments results to CSV file
with open(BASELINE_EXPERIMENTS, 'w') as f:
    writer = csv.writer(f)
    for result in run_results:
        # write the data
        writer.writerow(result)

f.close()


## Geração das imagens a partir dos arquivos .dot

In [5]:
!dot -Tpng ./Machine_Learning/DT/GINI_BASELINE.dot -o ./Machine_Learning/DT/GINI_BASELINE.png 
!dot -Tpng ./Machine_Learning/DT/ENTROPY_BASELINE.dot -o ./Machine_Learning/DT/ENTROPY_BASELINE.png 

# Experimento B

Abaixo se encontra o código utilizado para gerar os dados para o experimento A.
São produzidos os seguintes arquivos:

    * modified.csv - contém o resultado de todas execuções tanto para a implementação que utiliza a medida Gini quanto para a medida de entropia.

    * GINI_MODIFIED_X.png - contém a imagem da árvore de decisão do algoritmo.
    
    * GINI_BASELINE.png - contém a imagem da árvore de decisão da medida Gini. 

## Parâmetros de execução

Aqui podes definir os parâmetros para a execução dos experimentos

In [32]:
# Consts declaration
GINI_BASELINE_GRAPH_FILE = "./Machine_Learning/DT/GINI_BASELINE.dot"
OUTPUT_PATH = "./Machine_Learning/DT/"
MODIFIED_EXPERIMENTS = "./Machine_Learning/DT/modified.csv"
NUM_OF_RUNS = 100
MAX_DEPTH = [5, 10]
MIN_SAMPLES_AT_LEAF = [5, 10]

## Execução dos experimentos e exportação para arquivo CSV

In [36]:
# Runs the data the defined number of times and saves the run results
combined_parameters = []
for d in MAX_DEPTH:
    for s in MIN_SAMPLES_AT_LEAF:
        combined_parameters.append([d, s])

header = ["Gini"]
for comb in combined_parameters:
    header.append("Gini Modified D{0} S{1}".format(comb[0], comb[1]))
run_results = [header]

for i in range(NUM_OF_RUNS):
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(x), np.array(y).squeeze(), train_size=0.8)
    
    classifier_gini = tree.DecisionTreeClassifier(criterion='gini')
    classifier_gini = classifier_gini.fit(X_train, Y_train)
    with open(GINI_BASELINE_GRAPH_FILE, 'w') as out_file:
        dot_data = tree.export_graphviz(classifier_gini, feature_names=x.columns, out_file=out_file)
    
    classifiers = list(map(lambda c : tree.DecisionTreeClassifier(criterion='gini', max_depth=c[0], min_samples_leaf=c[1]),
                           combined_parameters))

    classifiers = [c.fit(X_train, Y_train) for c in classifiers]

    count = 0
    for c in classifiers:
        output_file = OUTPUT_PATH + 'GINI_MODIFIED_{}'.format(count) + '.dot'
        with open(output_file, 'w') as out_file:
            dot_data = tree.export_graphviz(c, feature_names=x.columns, out_file=out_file)
        count += 1
    
    gini_score = classifier_gini.score(X_test, Y_test)
    modified_score = [ x.score(X_test, Y_test) for x in classifiers ]

    current_results = [gini_score]
    current_results += modified_score

    run_results.append(current_results)  

# Save experiments results to CSV file
with open(MODIFIED_EXPERIMENTS, 'w') as f:
    writer = csv.writer(f)
    for result in run_results:
        # write the data
        writer.writerow(result)

f.close()


## Geração das imagens a partir dos arquivos .dot

In [37]:
!dot -Tpng ./Machine_Learning/DT/GINI_BASELINE.dot -o ./Machine_Learning/DT/GINI_BASELINE.png 
!dot -Tpng ./Machine_Learning/DT/GINI_MODIFIED_0.dot -o ./Machine_Learning/DT/GINI_MODIFIED_0.png 
!dot -Tpng ./Machine_Learning/DT/GINI_MODIFIED_1.dot -o ./Machine_Learning/DT/GINI_MODIFIED_1.png 
!dot -Tpng ./Machine_Learning/DT/GINI_MODIFIED_2.dot -o ./Machine_Learning/DT/GINI_MODIFIED_2.png 
!dot -Tpng ./Machine_Learning/DT/GINI_MODIFIED_3.dot -o ./Machine_Learning/DT/GINI_MODIFIED_3.png 